# Deduplicatie / ontdubbelen

De basis van ontdubbelen van data is vrij simpel: rijen die meerdere keren voorkomen, moeten verwijderd. Dit gebeurt bijvoorbeeld veel in het trainen van ML modellen, maar dit is niet waar dit notebook over gaat. Als je daar wél in geïnteresseerd bent, zie dan [pandas.DataFrame.drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

In ETL en veel andere processen is echter vaak een "fuzzy matching" nodig van teksten die op elkaar lijken, bijvoorbeeld

* Licht andere schrijfwijzen van namen of adressen
* Padnamen
* Producten
* Etc.

Eén van de meestgebruikte algoritmen om te bepalen of namen en teksten op elkaar lijken is het berekenen van een Levenshtein-afstand. Hier kan de Python-library [`thefuzz`](https://github.com/seatgeek/thefuzz) je bijvoorbeeld bij helpen.

In [2]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from thefuzz import fuzz
from thefuzz import process

## Gebruik

In de basis berekent `thefuzz` de "Levenshtein-afstand": een score waarbij 100 twee identieke teksten aanduiden:

In [3]:
origineletekst = "Deze tekst is gelijk aan de andere tekst"
vergelijktekst = origineletekst
fuzz.ratio(origineletekst, vergelijktekst)

100

Hoe meer je afwijkt, hoe lager de score wordt:

In [5]:
r1 = fuzz.ratio("Deze tekst is gelijk aan de andere tekst", "De andere tekst is gelijk aan deze tekst")
r2 = fuzz.ratio("Deze tekst is gelijk aan de andere tekst", "Deze tekst is gelijk aan de andere text")
r3 = fuzz.ratio("Deze tekst is gelijk aan de andere tekst", "Milliways is een restaurant aan het eind van het universum")
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

r1: 82, r2: 96, r3: 41


In [4]:
fuzz.ratio("Wortell Smart Learning", "Wortell Smart Learning B.V.")

90

In [7]:
# Werkt de simple ratio ongeacht leestekens?
fuzz.ratio("Wortell Smart Learning BV", "Wortell Smart Learning B.V.")

96

Er zijn diverse varianten, zoals:

* **Simple ratio** (de ratio hierboven: overeenkomst tussen twee teksten)
* **Partial ratio** (de ratio van de "most similar substring")
* **Token sort ratio** (sorteert eerst alvorens de "gelijkheid" te berekenen)

In [6]:
## Partial ratio
r1 = fuzz.partial_ratio("Deze tekst is gelijk aan de andere tekst", "De andere tekst is gelijk aan deze tekst")
r2 = fuzz.partial_ratio("Deze tekst is gelijk aan de andere tekst", "Deze tekst is gelijk aan de andere text")
r3 = fuzz.partial_ratio("Deze tekst is gelijk aan de andere tekst", "Milliways is een restaurant aan het eind van het universum")
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

r1: 85, r2: 95, r3: 45


In [5]:
fuzz.partial_ratio("Wortell Smart Learning", "Wortell Smart Learning B.V.")

100

In [7]:
## Token sort ratio
# Merk op dat de eerste tekst nu weer 100% identiek is.
r1 = fuzz.token_sort_ratio("Deze tekst is gelijk aan de andere tekst", "De andere tekst is gelijk aan deze tekst")
r2 = fuzz.token_sort_ratio("Deze tekst is gelijk aan de andere tekst", "Deze tekst is gelijk aan de andere text")
r3 = fuzz.token_sort_ratio("Deze tekst is gelijk aan de andere tekst", "Milliways is een restaurant aan het eind van het universum")
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

r1: 100, r2: 96, r3: 47


In [6]:
fuzz.token_sort_ratio("Wortell Limited Company", "Wortell Company (Limited)")

100

In [11]:
## Maar token-sort gaat ook fout als de woorden in een andere volgorde staan die echt iets anders betekenen!
rest1 = "Restaurant de Carnivoor. Hier eet je biefstuk, geen groente. Vegetariërs niet welkom."
rest2 = "Restaurant de Vegetariër. Hier eet je groente, geen biefstuk. Carnivoren niet welkom."
r1 = fuzz.token_sort_ratio(rest1, rest2)
print(r1)

98


## Process

Wanneer je een "master set" hebt, kun je hiermee op basis van de mogelijke keuzes bepalen welke het beste past.

In [12]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]

In [14]:
process.extract("new york jets", choices, limit=2)

[('New York Jets', 100), ('New York Giants', 79)]

In [15]:
# extractOne geeft één resultaat:
process.extractOne("cowboys", choices)

('Dallas Cowboys', 90)